# How to create a sensor

This tutorial demonstrates how to create a sensor.

There are different type of sensors available: irradiance sensor, radiance sensor, camera sensor.

## Prerequisites

### Perform imports

In [1]:
from pathlib import Path

In [2]:
from ansys.speos.core import GeoRef, Project, Speos, launcher
from ansys.speos.core.generic.parameters import (
    CameraSensorParameters,
    ColorParameters,
    PhotometricCameraParameters,
)
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.sensor import (
    Sensor3DIrradiance,
    SensorCamera,
    SensorIrradiance,
    SensorRadiance,
)

### Define constants

The constants help ensure consistency and avoid repetition throughout the example.

In [3]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
FILES = "CameraInputFiles"

### Define helper functions

In [4]:
def create_helper_geometries(project: Project):
    """Create bodies and faces."""

    def create_face(body):
        (
            body.create_face(name="TheFaceF")
            .set_vertices([0, 0, 0, 1, 0, 0, 0, 1, 0])
            .set_facets([0, 1, 2])
            .set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
            .commit()
        )

    root_part = project.create_root_part().commit()
    body_b1 = root_part.create_body(name="TheBodyB").commit()
    body_b2 = root_part.create_body(name="TheBodyC").commit()
    body_b3 = root_part.create_body(name="TheBodyD").commit()
    body_b4 = root_part.create_body(name="TheBodyE").commit()
    for b in [body_b1, body_b2, body_b3, body_b4]:
        create_face(b)

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [5]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Connect to the RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. The launch_local_speos_rpc_method can
be used to start a local instance of the service.

In [6]:
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


## Create a new project

The only way to create a sensor, is to create it from a project.

In [7]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


## Create

Create locally.

The mention "local: " is added when printing the sensor

In [8]:

distortion_file_path = str(assets_data_path / FILES / "CameraDistortion_130deg.OPTDistortion")
transmittance_file_path = str(assets_data_path / FILES / "CameraTransmittance.spectrum")
blue_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityBlue.spectrum")
green_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityGreen.spectrum")
red_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityRed.spectrum")

sensor1 = p.create_sensor(name="Camera.1", feature_type=SensorCamera)
sensor1.distortion_file_uri = distortion_file_path
# Choose photometric mode
sensor1.set_mode_photometric().transmittance_file_uri = transmittance_file_path
# Choose color mode (will imply to give spectrum file for blue, green and red)
mode_color = sensor1.photometric.set_mode_color()
mode_color.blue_spectrum_file_uri = blue_spectrum_path
mode_color.green_spectrum_file_uri = green_spectrum_path
mode_color.red_spectrum_file_uri = red_spectrum_path
print(sensor1)

local: {
    "name": "Camera.1",
    "description": "",
    "metadata": {},
    "sensor_guid": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wavelengths_range": {
                    "w_start": 400.0,
                    "w_end": 700.0,
        

## Push it to the server.

Now that it is committed to the server, the mention "local: " is no more present when printing the
sensor.

In [9]:
sensor1.commit()
print(sensor1)

{
    "name": "Camera.1",
    "metadata": {
        "UniqueId": "6c7bfc67-a116-48af-8b1a-f52ff286c1d7"
    },
    "sensor_guid": "57c8d026-2565-4f43-a511-3d689fd8e747",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wavelengths

## Another example

Set more characteristics, and use parameters classes.

Camera feature is created with the same default values as the GUI speos.

If the user would like to modify the camera characteristics,
it is possible to do so as below.

In [10]:
distortion_file_path = str(assets_data_path / FILES / "CameraDistortion_130deg.OPTDistortion")
transmittance_file_path = str(assets_data_path / FILES / "CameraTransmittance.spectrum")
blue_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityBlue.spectrum")
green_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityGreen.spectrum")
red_spectrum_path = str(assets_data_path / FILES / "CameraSensitivityRed.spectrum")

color_params = ColorParameters(
    blue_spectrum_file_uri=blue_spectrum_path,
    green_spectrum_file_uri=green_spectrum_path,
    red_spectrum_file_uri=red_spectrum_path,
)
photo_params = PhotometricCameraParameters(
    color_mode=color_params, layer_type="by_source", transmittance_file_uri=transmittance_file_path
)
param = CameraSensorParameters(
    sensor_type_parameters=photo_params,
    distortion_file_uri=distortion_file_path,
    axis_system=[20, 10, 40, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    # camera location [Origin, Xvector, Yvector, Zvector]
    focal_length=5.5,
    height=6,  # dimensions
    width=6,
)

sensor2 = p.create_sensor("Camera_Parameter", feature_type=SensorCamera, parameters=param)
sensor2.commit()

print(sensor2)

{
    "name": "Camera_Parameter",
    "metadata": {
        "UniqueId": "3ee97b4f-567a-4985-885e-da425199300f"
    },
    "sensor_guid": "efb18449-0d89-4b50-af8c-17af4c43f9e6",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wav

## Read

### Sensor Instance

A mention "local: " is added if it is not yet committed to the server

In [11]:
print(sensor1)

{
    "name": "Camera.1",
    "metadata": {
        "UniqueId": "6c7bfc67-a116-48af-8b1a-f52ff286c1d7"
    },
    "sensor_guid": "57c8d026-2565-4f43-a511-3d689fd8e747",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wavelengths

### Project

Committed feature will appear in the project.

In [12]:
print(p)

{
    "sensors": [
        {
            "name": "Camera.1",
            "metadata": {
                "UniqueId": "6c7bfc67-a116-48af-8b1a-f52ff286c1d7"
            },
            "sensor_guid": "57c8d026-2565-4f43-a511-3d689fd8e747",
            "description": "",
            "result_file_name": "",
            "sensor": {
                "camera_sensor_template": {
                    "sensor_mode_photometric": {
                        "acquisition_integration": 0.01,
                        "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                        "gamma_correction": 2.2,
                        "png_bits": "PNG_16",
                        "color_mode_color": {
                            "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                            "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                            "blue_spe

## Update

Tipp: if you are manipulating a sensor already committed, don't forget to commit your changes.

If you don't, you will still only watch what is committed on the server.

In [13]:
# modify f number and axis system
sensor1.f_number = 11
sensor1.axis_system = [17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1]
sensor1.commit()
print(sensor1)

{
    "name": "Camera.1",
    "metadata": {
        "UniqueId": "6c7bfc67-a116-48af-8b1a-f52ff286c1d7"
    },
    "sensor_guid": "57c8d026-2565-4f43-a511-3d689fd8e747",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wavelengths

## Reset

Possibility to reset local values from the one available in the server.

In [14]:
sensor1.set_mode_geometric()  # set camera in geometric mode but no commit
sensor1.reset()  # reset -> this will apply the server value (photometric mode) to the local value
sensor1.delete()  # delete (to display the local value with the below print)
print(sensor1._sensor_template)

ansys.api.speos.sensor.v1.SensorTemplate
{
    "camera_sensor_template": {
        "sensor_mode_photometric": {
            "acquisition_integration": 0.01,
            "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
            "gamma_correction": 2.2,
            "png_bits": "PNG_16",
            "color_mode_color": {
                "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                "balance_mode_none": {}
            },
            "wavelengths_range": {
                "w_start": 400.0,
                "w_end": 700.0,
                "w_sampling": 13
            },
            "acquisition_lag_time": 0.0
        },
        "focal_length": 5.0,
        "imager_distance": 10.0,
       

## Delete

Once the data is deleted from the server, you can still work with local data and maybe commit
later.

In [15]:
sensor2.delete()
print(sensor2)

local: {
    "name": "Camera_Parameter",
    "description": "",
    "metadata": {},
    "sensor_guid": "",
    "result_file_name": "",
    "sensor": {
        "camera_sensor_template": {
            "sensor_mode_photometric": {
                "acquisition_integration": 0.01,
                "transmittance_file_uri": "/app/assets/CameraInputFiles/CameraTransmittance.spectrum",
                "gamma_correction": 2.2,
                "png_bits": "PNG_16",
                "color_mode_color": {
                    "red_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityRed.spectrum",
                    "green_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityGreen.spectrum",
                    "blue_spectrum_file_uri": "/app/assets/CameraInputFiles/CameraSensitivityBlue.spectrum",
                    "balance_mode_none": {}
                },
                "wavelengths_range": {
                    "w_start": 400.0,
                    "w_end": 700.0,


In [16]:
sensor1.delete()

## Other sensors

### Irradiance sensor

In [17]:
sensor3 = p.create_sensor(name="Irradiance.1", feature_type=SensorIrradiance)
sensor3.commit()
print(sensor3)

{
    "name": "Irradiance.1",
    "metadata": {
        "UniqueId": "a8b15a58-43cd-4fbd-a6c6-32f146650f1e"
    },
    "sensor_guid": "c09702cc-79b5-47a1-a096-9e69f2aad550",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "irradiance_sensor_template": {
            "sensor_type_photometric": {},
            "illuminance_type_planar": {},
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ],
            "layer_type_none": {},
            "ray_file_type": "RayFileNone",
            "integ

In [18]:
sensor3.set_type_radiometric()
sensor3.set_layer_type_polarization()
sensor3.commit()
print(sensor3)

{
    "name": "Irradiance.1",
    "metadata": {
        "UniqueId": "a8b15a58-43cd-4fbd-a6c6-32f146650f1e"
    },
    "sensor_guid": "c09702cc-79b5-47a1-a096-9e69f2aad550",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "irradiance_sensor_template": {
            "sensor_type_radiometric": {},
            "illuminance_type_planar": {},
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ],
            "layer_type_polarization": {},
            "ray_file_type": "RayFileNone",
          

In [19]:
sensor3.delete()

### radiance sensor

In [20]:
sensor4 = p.create_sensor(name="Radiance.1", feature_type=SensorRadiance)
sensor4.commit()
print(sensor4)

{
    "name": "Radiance.1",
    "metadata": {
        "UniqueId": "e635dbba-5883-4183-ac23-118b4658d457"
    },
    "sensor_guid": "7ec5b789-c230-4689-a8f2-02c534e23642",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "radiance_sensor_template": {
            "sensor_type_photometric": {},
            "focal": 250.0,
            "integration_angle": 5.0,
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ],
            "layer_type_none": {},
            "observer_point": []
        },

In [21]:
sensor4.focal = 200
sensor4.set_type_spectral()
sensor4.set_layer_type_source()
sensor4.commit()
print(sensor4)

{
    "name": "Radiance.1",
    "metadata": {
        "UniqueId": "e635dbba-5883-4183-ac23-118b4658d457"
    },
    "sensor_guid": "7ec5b789-c230-4689-a8f2-02c534e23642",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "radiance_sensor_template": {
            "sensor_type_spectral": {
                "wavelengths_range": {
                    "w_start": 400.0,
                    "w_end": 700.0,
                    "w_sampling": 13
                }
            },
            "focal": 200.0,
            "integration_angle": 5.0,
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0

In [22]:
sensor4.delete()
print(sensor4)

local: {
    "name": "Radiance.1",
    "description": "",
    "metadata": {},
    "sensor_guid": "",
    "result_file_name": "",
    "sensor": {
        "radiance_sensor_template": {
            "sensor_type_spectral": {
                "wavelengths_range": {
                    "w_start": 400.0,
                    "w_end": 700.0,
                    "w_sampling": 13
                }
            },
            "focal": 200.0,
            "integration_angle": 5.0,
            "dimensions": {
                "x_start": -50.0,
                "x_end": 50.0,
                "x_sampling": 100,
                "y_start": -50.0,
                "y_end": 50.0,
                "y_sampling": 100
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
        

### 3D irradiance sensor

In [23]:
create_helper_geometries(p)
sensor5 = p.create_sensor(name="3D_Irradiance.2", feature_type=Sensor3DIrradiance)
sensor5.geometries = [GeoRef.from_native_link("TheBodyB/TheFaceF")]
sensor5.commit()
print(sensor5)

{
    "name": "3D_Irradiance.2",
    "metadata": {
        "UniqueId": "078ad64c-dd65-4cec-90a2-31c9c8bdfbbf"
    },
    "sensor_guid": "27586610-d299-4bcc-86b1-6c950a7de422",
    "description": "",
    "result_file_name": "",
    "sensor": {
        "name": "3D_Irradiance.2",
        "irradiance_3d": {
            "type_photometric": {
                "integration_type_planar": {
                    "reflection": true,
                    "transmission": true,
                    "absorption": true
                }
            },
            "layer_type_none": {},
            "geometries": {
                "geo_paths": [
                    "TheBodyB/TheFaceF"
                ]
            },
            "ray_file_type": "RayFileNone"
        },
        "description": "",
        "metadata": {}
    }
}


In [24]:
speos.close()

True